In [39]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as st
import copy



##### Для начала сделаем анализ для каждого штата, дла каждого гендера одновременно

In [63]:
# Загрузка данных из файла Excel
file_path = 'Автострахование.xlsx'
df = pd.read_excel(file_path)
df

,STATE,CLASS,GENDER,AGE,PAID
0,STATE 14,C6,M,97,1134.44
1,STATE 15,C6,M,96,3761.24
2,STATE 15,C11,M,95,7842.31
3,STATE 15,F6,F,95,2384.67
4,STATE 15,F6,M,95,650.00
...,...,...,...,...,...
6768,STATE 15,C71,M,50,39.31
6769,STATE 02,C71,M,50,35.00
6770,STATE 17,C7B,F,50,27.46
6771,STATE 01,C71,F,50,25.00


In [64]:
# Разделение данных на группы по полу и географическому критерию
grouped_data = data.groupby(['GENDER', 'STATE'])

In [65]:
# Расчет мер риска для каждой группы
risk_measures = grouped_data['PAID'].agg([
    np.var,                           # Дисперсия
    np.std,                           # Стандартное отклонение (СКО)
    lambda x: np.var(x[x >= 0]),      # Положительная полудисперсия
    lambda x: np.var(x[x < 0]),       # Отрицательная полудисперсия
    lambda x: np.median(np.abs(x - np.median(x))),  # Среднее абсолютное отклонение
    np.max,                           # Максимальный ущерб
    lambda x: np.quantile(x, 0.95)    # 95%-ная квантиль (VaR)
])

In [66]:
# Вывод результатов
print(risk_measures)

                          var          std    <lambda_0>  <lambda_1>  \
GENDER STATE                                                           
F      STATE 01  3.017817e+06  1737.186552  2.958644e+06         NaN   
       STATE 02  5.237317e+06  2288.518475  5.225277e+06         NaN   
       STATE 03  1.268172e+07  3561.139769  1.258708e+07         NaN   
       STATE 04  4.378821e+06  2092.563259  4.360188e+06         NaN   
       STATE 06  8.129335e+06  2851.198821  8.095321e+06         NaN   
       STATE 07  3.476531e+07  5896.211520  3.446300e+07         NaN   
       STATE 10  6.725159e+06  2593.291076  6.667679e+06         NaN   
       STATE 11  2.599745e+06  1612.372421  2.079796e+06         NaN   
       STATE 12  8.093093e+06  2844.836191  8.006996e+06         NaN   
       STATE 13  1.530977e+07  3912.770266  1.510833e+07         NaN   
       STATE 14  4.035363e+06  2008.821271  3.947638e+06         NaN   
       STATE 15  5.087018e+06  2255.441951  5.080815e+06        

In [67]:
result_df = pd.DataFrame({
    'Дисперсия': risk_measures['var'],
    'Стандартное отклонение (СКО)': risk_measures['std'],
    'Положительная полудисперсия': risk_measures['<lambda_0>'],
    'Отрицательная полудисперсия': risk_measures['<lambda_1>'],
    'Среднее абсолютное отклонение': risk_measures['<lambda_2>'],
    'Максимальный ущерб': risk_measures['amax'],
    '95%-ная квантиль (VaR)': risk_measures['<lambda_3>']
})

result_df

Дисперсия  Стандартное отклонение (СКО)  \
GENDER STATE                                                  
F      STATE 01  3.017817e+06                   1737.186552   
       STATE 02  5.237317e+06                   2288.518475   
       STATE 03  1.268172e+07                   3561.139769   
       STATE 04  4.378821e+06                   2092.563259   
       STATE 06  8.129335e+06                   2851.198821   
       STATE 07  3.476531e+07                   5896.211520   
       STATE 10  6.725159e+06                   2593.291076   
       STATE 11  2.599745e+06                   1612.372421   
       STATE 12  8.093093e+06                   2844.836191   
       STATE 13  1.530977e+07                   3912.770266   
       STATE 14  4.035363e+06                   2008.821271   
       STATE 15  5.087018e+06                   2255.441951   
       STATE 17  6.640458e+06                   2576.908577   
M      STATE 01  4.361947e+06                   2088.527462   
       STATE 02  5.592754e+06                   2364.900509   
       STATE 03  6.565808e+06                   2562.383307   
       STATE 04  8.918667e+06                   2986.413730   
       STATE 06  4.589474e+06                   2142.305783   
       STATE 07  3.574316e+06                   1890.586041   
       STATE 10  4.327433e+06                   2080.248218   
       STATE 11  2.178443e+06                   1475.955026   
       STATE 12  6.667749e+06                   2582.198478   
       STATE 13  6.731985e+06                   2594.607001   
       STATE 14  8.017739e+06                   2831.561202   
       STATE 15  5.714451e+06                   2390.491849   
       STATE 17  1.627670e+07                   4034.438665   

                 Положительная полудисперсия  Отрицательная полудисперсия  \
GENDER STATE                                                                
F      STATE 01                 2.958644e+06                          NaN   
       STATE 02                 5.225277e+06                          NaN   
       STATE 03                 1.258708e+07                          NaN   
       STATE 04                 4.360188e+06                          NaN   
       STATE 06                 8.095321e+06                          NaN   
       STATE 07                 3.446300e+07                          NaN   
       STATE 10                 6.667679e+06                          NaN   
       STATE 11                 2.079796e+06                          NaN   
       STATE 12                 8.006996e+06                          NaN   
       STATE 13                 1.510833e+07                          NaN   
       STATE 14                 3.947638e+06                          NaN   
       STATE 15                 5.080815e+06                          NaN   
       STATE 17                 6.609572e+06                          NaN   
M      STATE 01                 4.324017e+06                          NaN   
       STATE 02                 5.584614e+06                          NaN   
       STATE 03                 6.535127e+06                          NaN   
       STATE 04                 8.897974e+06                          NaN   
       STATE 06                 4.577491e+06                          NaN   
       STATE 07                 3.551106e+06                          NaN   
       STATE 10                 4.300216e+06                          NaN   
       STATE 11                 1.633832e+06                          NaN   
       STATE 12                 6.624169e+06                          NaN   
       STATE 13                 6.680986e+06                          NaN   
       STATE 14                 7.952554e+06                          NaN   
       STATE 15                 5.710249e+06                          NaN   
       STATE 17                 1.621772e+07                          NaN   

                 Среднее абсолютное отклонение  Максимальный ущерб  \
GENDER STATE                

##### Теперь сделаем отдельно

In [86]:
# создаем функцию:
def semivar(data):
    # считаем среднее значение:
    mu = np.mean(data)
    # создаем массивы отдельно положительных и отрицательных отклонений от среднего в каждой строке датасета:
    pos = np.maximum(data - mu, 0)
    neg = np.maximum(mu - data, 0)
    # рассчитываем положительную и отрицательную полудисперсии:
    pos_var = np.mean(pos ** 2)
    neg_var = np.mean(neg ** 2)
    # возвращаем рассчиатнные значения:
    return pos_var, neg_var

In [87]:
# создаем сводную таблицу:
groups = df.pivot_table(index='gender', values='paid', 
                        aggfunc=['var', 
                                 'std', 
                                 lambda x: semivar(x)[0], 
                                 lambda x: semivar(x)[1], 
                                 'max', 
                                 lambda x: np.percentile(x, 95)]).reset_index()
# переименовываем столбцы:
groups.columns = ['Пол', 'Дисперсия', 'СКО', 'Положительная полудисперсия', 
                  'Отрицательная полудисперсия', 'Максимальный ущерб', 'Рисковый капитал']
# выводим сводную таблицу:
groups

,Пол,Дисперсия,СКО,Положительная полудисперсия,Отрицательная полудисперсия,Максимальный ущерб,Рисковый капитал
0,F,7.620810e+06,2760.581523,6.603872e+06,1.013987e+06,60000.00,6448.3935
1,M,6.629054e+06,2574.694894,5.696383e+06,9.310888e+05,59113.78,6303.4650


##### Исходя из представленных данных, можно сделать некоторые выводы:
1. В результате расчета различных мер риска, женщины оказались более рисковой группой. 
2. Однако, с учетом контекста автострахования, лучшей мерой риска для выделенных групп, а также в общем контексте, будет рисковый капитал. Эта мера лучше всего объясняет возможные потери с вероятностью 95%, не страдает так сильно от выбросов, поскольку представляет собой квантиль, и отличается от оценок, основанных на средних значениях, таких как дисперсия, стандартное отклонение и полудисперсии.

In [88]:
# создаем функцию:
def semivar(data):
    # считаем среднее значение:
    mu = np.mean(data)
    # создаем массивы отдельно положительных и отрицательных отклонений от среднего в каждой строке датасета:
    pos = np.maximum(data - mu, 0)
    neg = np.maximum(mu - data, 0)
    # рассчитываем положительную и отрицательную полудисперсии:
    pos_var = np.mean(pos ** 2)
    neg_var = np.mean(neg ** 2)
    # возвращаем рассчиатнные значения:
    return pos_var, neg_var

In [89]:
# создаем сводную таблицу:
groups = df.pivot_table(index='state', values='paid', 
                        aggfunc=['var', 
                                 'std', 
                                 lambda x: semivar(x)[0], 
                                 lambda x: semivar(x)[1], 
                                 'max', 
                                 lambda x: np.percentile(x, 95)]).reset_index()
# переименовываем столбцы:
groups.columns = ['Штат', 'Дисперсия', 'СКО', 'Положительная полудисперсия', 
                  'Отрицательная полудисперсия', 'Максимальный ущерб', 'Рисковый капитал']
# выводим сводную таблицу:
groups

,Штат,Дисперсия,СКО,Положительная полудисперсия,Отрицательная полудисперсия,Максимальный ущерб,Рисковый капитал
0,STATE 01,3.941779e+06,1985.391511,3.279568e+06,6.384659e+05,15845.00,4462.1750
1,STATE 02,5.450391e+06,2334.607321,4.584938e+06,8.605951e+05,23231.54,5837.7055
2,STATE 03,8.952699e+06,2992.106079,7.839452e+06,1.087521e+06,23344.38,7470.8455
3,STATE 04,7.332263e+06,2707.815109,6.462816e+06,8.584376e+05,28593.49,6041.3425
4,STATE 06,5.947683e+06,2438.787106,4.771203e+06,1.166917e+06,19348.67,7273.6560
5,STATE 07,1.694645e+07,4116.606786,1.571886e+07,1.164593e+06,60000.00,6414.9680
6,STATE 10,5.324949e+06,2307.585095,4.371115e+06,9.345403e+05,21404.96,6000.6250
7,STATE 11,2.141788e+06,1463.484925,1.243637e+06,6.601751e+05,4635.00,3941.1440
8,STATE 12,7.211720e+06,2685.464616,5.561995e+06,1.620528e+06,16161.53,8471.5250
9,STATE 13,9.847573e+06,3138.084260,8.670503e+06,1.129726e+06,33137.50,6744.7880


#### Исходя из представленных данных, можно сделать некоторые выводы:

1. Штат 07 имеет наибольшую дисперсию и стандартное отклонение (СКО), что может указывать на более значительные различия в уровне ущерба.
2. Положительная полудисперсия также выше у штата 07, что указывает на более высокие потери в данной категории ущерба.
3. Максимальный ущерб также превышает другие штаты в штате 07.
4. Рисковый капитал в штате 12 также является одним из высоких, что означает больший показатель уровня риска.

### Значит:

##### 1. Штаты с наибольшим риском: 
Наиболее рисковыми штатами можно считать 12 и 07, где штат 07 в большинстве случаев выступает в роли лидера по мерам риска.

##### 2. Предпочтение в использовании меры риска: 
Отдать предпочтение можно рисковому капиталу из-за его универсальности и простоты применения. Однако, следует отметить, что дисперсия и стандартное отклонение (СКО) не всегда являются надежными мерами риска, а полудисперсии могут давать различные оценки.

##### 3. Максимальное значение риска:
Оценка риска на основе максимальных значений имеет право на существование, однако она может быть не слишком объективной и подвержена влиянию выбросов.

##### 4. 95%-ная квантиль (VaR): 
Квантиль, основанная на 95% уровне, более устойчива к выбросам и обладает гибкой политикой оценки риска.

Таким образом, для более точной и надежной оценки риска рекомендуется использовать несколько мер, включая рисковый капитал и квантиль, чтобы учесть различные аспекты и сделать более обоснованный вывод о степени риска для каждого штата.